# **Tiempo en el mercado**

In [3]:
# Manejo de datos
import os # Directorios
import pandas as pd # Manipulación df
# Gráficas
import plotly.graph_objects as go #Para obtener librería usar: pip install plotly
from plotly.subplots import make_subplots
import plotly.io as pio # Exportar gráfica

# Obtener el directorio actual de trabajo
directorio_actual = os.getcwd()

# Directorio donde se encuentran los archivos JSON (ruta relativa)
directorio_json = os.path.join(directorio_actual, '../../db/datos_json')

# Obtener la lista de archivos JSON en el directorio
archivos_json = os.listdir(directorio_json)

# Cargar los archivos JSON y crear DataFrames
for archivo in archivos_json:
    nombre_tabla = archivo.replace('datos_', '').replace('.json', '')
    ruta_json = os.path.join(directorio_json, archivo)
    globals()[f"df_{nombre_tabla}"] = pd.read_json(ruta_json)

# Obtener todos los nombres de las variables globales
nombres_variables_globales = list(globals().keys())

# Filtrar los nombres que comienzan con "df_", contienen "alfa_q" y "pachuca"
nombres_df_filtrados = [
    nombre for nombre in nombres_variables_globales 
    # Caso de cuando no son las alfa q
    if nombre.startswith("df_") and "tiempo" in nombre and "tulancingo" in nombre
]

# Imprimir la lista de DataFrames filtrados
print("Lista de DataFrames filtrados:")
nombres_df_filtrados

Lista de DataFrames filtrados:


['df_tiempo_dias_tulancingo']

In [4]:
df_tiempo_dias_tulancingo.rename(columns={'Tiempo_publicacion':'tiempo_de_publicacion','tiempo_publicacion':'tiempo_de_publicacion'}, inplace=True)

# 1. **Tiempo de días en mercado**

In [5]:
df_mercado_pachuca = df_tiempo_dias_tulancingo[['id','tiempo_de_publicacion']]
df_mercado_pachuca

,id,tiempo_de_publicacion
0,1,23
1,2,18
2,3,45
3,4,23
4,5,23
...,...,...
82,83,23
83,84,22
84,85,22
85,86,23


In [6]:
print(df_mercado_pachuca.shape)
tiempo = df_mercado_pachuca['tiempo_de_publicacion'].unique()
print(tiempo)

(87, 2)
[23 18 45 22 50]


## Asignar rangos

In [7]:
# Definir la función de categorización
def categorizar(valor):
    if isinstance(valor, (int, float)):  # Verificar si el valor es un número (int o float)
        if 0 <= valor <= 10:
            return '0-10'
        elif 11 <= valor <= 30:
            return '11-30'
        elif 31 <= valor <= 100:
            return '31-100'
        elif 101 <= valor <= 364:
            return '101-364'
        else:
            return '>=365'
    elif isinstance(valor, str) and valor.isdigit():  # Verificar si el valor es una cadena numérica
        valor = int(valor)  # Convertir a entero si es un número
        if 0 <= valor <= 10:
            return '0-10'
        elif 11 <= valor <= 30:
            return '11-30'
        elif 31 <= valor <= 100:
            return '31-100'
        elif 101 <= valor <= 364:
            return '101-364'
        else:
            return '>=365'
    else:
        return 'No numérico'  # Manejar valores no numéricos

# Lista de nombres de los DataFrames
dataframes_name = ['df_mercado_pachuca']

# Lista de DataFrames
dataframes_list = [df_mercado_pachuca]

# Definir el orden de las categorías
todas_categorias = ['0-10', '11-30', '31-100', '101-364', '>=365']

# Diccionario para almacenar los DataFrames de conteo por nombre de DataFrame original
dataframes_conteo_por_nombre = {}

# Iterar sobre cada DataFrame
for name, df in zip(dataframes_name, dataframes_list):
    # Aplicar la función de categorización
    df = df.copy()
    df['dias'] = df['tiempo_de_publicacion'].apply(categorizar)
    # Contar el número de registros únicos por cada valor único en la columna 'dias'
    conteo_por_dias = df['dias'].value_counts().reindex(todas_categorias, fill_value=0)
    # Crear un nuevo DataFrame de conteo
    df_conteo = pd.DataFrame({'dias': conteo_por_dias.index, 'conteo': conteo_por_dias.values})
    # Ordenar el DataFrame según el orden definido en todas_categorias
    df_conteo['dias'] = pd.Categorical(df_conteo['dias'], categories=todas_categorias, ordered=True)
    df_conteo = df_conteo.sort_values(by='dias')
    # Almacenar el DataFrame de conteo en el diccionario por nombre de DataFrame original
    dataframes_conteo_por_nombre[name] = df_conteo
    # Imprimir el nombre del DataFrame de conteo y su contenido
    nuevo_nombre_df_conteo = 'conteo_' + name
    globals()[nuevo_nombre_df_conteo] = df_conteo
    print(f'Nombre del DataFrame de conteo: {nuevo_nombre_df_conteo}')
    print(df_conteo)

Nombre del DataFrame de conteo: conteo_df_mercado_pachuca
      dias  conteo
0     0-10       0
1    11-30      63
2   31-100      24
3  101-364       0
4    >=365       0


In [8]:
conteo_df_mercado_pachuca

,dias,conteo
0,0-10,0
1,11-30,63
2,31-100,24
3,101-364,0
4,>=365,0


In [9]:
values = conteo_df_mercado_pachuca['conteo']
labels = conteo_df_mercado_pachuca['dias']
colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6']

fig = go.Figure()
fig.add_trace(go.Bar(
    x=labels, 
    y=values, 
    marker_color=colores,  # Especifica los colores de las barras
    text=values,  # Texto que se mostrará en las barras 
    textposition='inside',  # Posición del texto (puede ser 'inside' o 'outside')
))
fig.update_layout(
    #title='Número de unidades por días en el mercado',  # Título de la gráfica
    xaxis=dict(title='Rango días'),  # Título del eje x
    yaxis=dict(
        title='Propiedades', # Título del eje y
        gridcolor='#dddcda', # Color de las líneas que dividen los rangos del eje Y
         gridwidth=1,
             ),  margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)    
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
)
# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets\graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')
guardar_grafico_como_html(fig, 'g_bar_tiempo_mercado', carpeta='assets\graficas')
fig.show()

___
## Directos

In [10]:
datos_tiempomercado_tulancingo = {
    'dias': ['0-10','11-30','31-100','101-364','>=365'],
    'conteo': [25,50,79,47,30]
}
tiempo_mercado_tulancingo_direct = pd.DataFrame(datos_tiempomercado_tulancingo)
tiempo_mercado_tulancingo_direct

,dias,conteo
0,0-10,25
1,11-30,50
2,31-100,79
3,101-364,47
4,>=365,30


In [11]:
values = tiempo_mercado_tulancingo_direct['conteo']
labels = tiempo_mercado_tulancingo_direct['dias']
colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6']

fig = go.Figure()
fig.add_trace(go.Bar(
    x=labels, 
    y=values, 
    marker_color=colores,  # Especifica los colores de las barras
    text=values,  # Texto que se mostrará en las barras 
    textposition='inside',  # Posición del texto (puede ser 'inside' o 'outside')
))
fig.update_layout(
    #title='Número de unidades por días en el mercado',  # Título de la gráfica
    xaxis=dict(title='Rango días'),  # Título del eje x
    yaxis=dict(
        title='Propiedades', # Título del eje y
        gridcolor='#dddcda', # Color de las líneas que dividen los rangos del eje Y
         gridwidth=1,
             ),  margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)    
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
)
# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets\graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')
guardar_grafico_como_html(fig, 'g_bar_tiempo_mercado', carpeta='assets\graficas')
fig.show()

# 2. **RIESGO**

In [12]:
conteo_df_mercado_pachuca

,dias,conteo
0,0-10,0
1,11-30,63
2,31-100,24
3,101-364,0
4,>=365,0


In [13]:
# Definir los valores para la columna 'stock'
stock_values = []
for dias in conteo_df_mercado_pachuca['dias']:
    if dias == '>=365':
        stock_values.append('Más de un año en stock')
    else:
        stock_values.append('Menos de un año en stock')

# Crear el DataFrame 'riesgo'
riesgo = pd.DataFrame({
    'stock': stock_values,
    'casas': conteo_df_mercado_pachuca['conteo']
})

# Agrupar por la columna 'stock' y sumar los valores de 'casas'
riesgo_stock = riesgo.groupby('stock').sum().reset_index()
# Ordenar el DataFrame para que 'Menos de un año en stock' quede abajo
riesgo_stock = riesgo_stock.sort_values(by='stock', ascending=False)
riesgo_stock

,stock,casas
1,Más de un año en stock,0
0,Menos de un año en stock,87


In [14]:
labels = riesgo_stock['stock']
values = riesgo_stock['casas']
colores = ['#9500ff', '#ff0059']
# Crear el gráfico de pastel
fig = go.Figure(data=[go.Pie(
    labels=labels, 
    values=values, 
    sort=False,    
    marker=dict(colors=colores,)
)])
fig.update_layout(margin=dict(l=10, r=10, t=10, b=10),)
# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets\graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')
guardar_grafico_como_html(fig, 'g_pie_riesgo', carpeta='assets\graficas')
fig.show()

## Directo

In [15]:
datos = {
    'stock': ['Más de un año en stcok','Menos de un año en stock'],
    'casas': [ 413, 2067 ]
}
# Crear DataFrame
df_riesgo_pachuca = pd.DataFrame(datos)
df_riesgo_pachuca

,stock,casas
0,Más de un año en stcok,413
1,Menos de un año en stock,2067


In [16]:
labels = df_riesgo_pachuca['stock']
values = df_riesgo_pachuca['casas']
colores = ['#9500ff', '#ff0059']
# Crear el gráfico de pastel
fig = go.Figure(data=[go.Pie(
    labels=labels, 
    values=values, 
    sort=False,    
    marker=dict(colors=colores,)
)])
fig.update_layout(margin=dict(l=10, r=10, t=10, b=10),)
# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets\graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')
guardar_grafico_como_html(fig, 'g_pie_riesgo', carpeta='assets\graficas')
fig.show()